In [14]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU, Convolution1D,Embedding,Convolution2D
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

In [15]:
# fix random seed for reproducibility
seed = 123
numpy.random.seed(seed)

In [16]:
# load dataset
dataframe = pd.read_csv("C:\\Users\\Sai\\Desktop\\Insofe\\exams\\PHD\\MLP_data2.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[1:,0:20].astype(float)
Y = dataset[1:,-1]

In [17]:
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [18]:
encoded_Y.shape

(3156,)

In [19]:
X.shape

(3156, 20)

In [20]:
X_train_r = numpy.zeros((len(X), 20, 1))
X_train_r[:, :, 0] = X[:, :20]
X_train_r

array([[[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 0.]],

       [[ 0.],
        [ 0.],
        [ 1.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       [[ 0.],
        [ 1.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       ..., 
       [[ 0.],
        [ 0.],
        [ 1.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       [[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       [[ 0.],
        [ 0.],
        [ 1.],
        ..., 
        [ 0.],
        [ 1.],
        [ 0.]]])

In [29]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X_train_r, encoded_Y):
    # create model
    model = Sequential()
    #model.add(Embedding(54,54,input_length=54))
    #model.add(Conv1D(activation='relu', input_shape=(54,), strides=1, filters=8, kernel_size=8))
    model.add(Convolution1D(nb_filter=128, filter_length=1, input_shape=(20, 1)))
    model.add(Dropout(0.3))
    model.add(Conv1D(activation='relu', strides=1, filters=128, kernel_size=8))
    model.add(Dropout(0.2))
    model.add(Conv1D(activation='relu', strides=1, filters=32, kernel_size=8))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Flatten())
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train_r[train], encoded_Y[train], epochs=20, batch_size=128, verbose=1)
    # evaluate the model
    scores = model.evaluate(X_train_r[test], encoded_Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

C:\Users\Sai\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(20, 1), filters=128, kernel_size=1)`
  if sys.path[0] == '':


Epoch 1/20
2103/2103 [==============================] - 7s 3ms/step - loss: 0.6819 - acc: 0.6396
Epoch 2/20
2103/2103 [==============================] - 2s 921us/step - loss: 0.5824 - acc: 0.7175
Epoch 3/20
2103/2103 [==============================] - 4s 2ms/step - loss: 0.5216 - acc: 0.7494
Epoch 4/20
2103/2103 [==============================] - 1s 530us/step - loss: 0.4572 - acc: 0.7941
Epoch 5/20
2103/2103 [==============================] - 1s 456us/step - loss: 0.3973 - acc: 0.8364
Epoch 6/20
2103/2103 [==============================] - 1s 433us/step - loss: 0.3824 - acc: 0.8421
Epoch 7/20
2103/2103 [==============================] - 1s 420us/step - loss: 0.3583 - acc: 0.8659
Epoch 8/20
2103/2103 [==============================] - 1s 470us/step - loss: 0.3574 - acc: 0.8569
Epoch 9/20
2103/2103 [==============================] - 1s 444us/step - loss: 0.3403 - acc: 0.8654
Epoch 10/20
2103/2103 [==============================] - 1s 461us/step - loss: 0.3449 - acc: 0.8654
Epoch 11/20
2